In [1]:
import cv2
import numpy as np
import sys
import os
import math

# 턱관절 운동범위 및 치아의 고저 측정

In [6]:
def call_mouse(event,x,y,flags,param):
    global clilk,idx,last_x1,last_x2,last_y1,last_y2
    
    if event==cv2.EVENT_LBUTTONDOWN:
#         oldx,oldy=x,y
        
        clilk=not clilk
        idx+=1
#         circle(img, center, radius, color[, thickness[, lineType[, shift]]]) -> img
        cv2.circle(frame,(x,y),5,(255,180,0),-1,cv2.LINE_AA)
        cv2.imshow('frame',frame)
        if idx==1:
            last_x1=x
            last_y1=y
                       
        if idx==2:
            last_x2=x
            last_y2=y
            
        if idx==3:
            cv2.line(frame,(last_x1,last_y1),(last_x2,last_y2),(255,180,0),5,cv2.LINE_AA)
            cv2.line(frame,(last_x1,last_y1),(x,y),(255,180,0),5,cv2.LINE_AA)
            #             atan( (점1 Y - 점2 Y) / ( 점1 X- 점2X)) - atan( (점 3 Y - 점2Y) / (점3 X - 점2X))  * 180/pi
            rad=math.atan((last_y2-last_y1)/(last_x2-last_x1))-math.atan((y-last_y1)/(x-last_x1))
            deg=(rad*180)/math.pi
#             text='벌린 입의 각도는'+' {0:0.2f}도 입니다.'.format(np.abs(deg))
            text='{0:0.2f} deg'.format(np.abs(deg))
            cv2.putText(frame,text,(last_x1+5,last_y1+20),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            cv2.imshow('frame',frame)
#             print(text)
        if idx==4:
            idx=0

#     elif event==cv2.EVENT_MOUSEMOVE:
#         if flags==cv2.EVENT_FLAG_LBUTTON:
#             cv2.line(frame,(oldx,oldy),(x,y),(0,0,0),10,cv2.LINE_AA)
#             cv2.imshow('frame',frame)
#             oldx,oldy=x,y
idx=0
clilk=True         
# oldx = oldy = -1 
cv2.namedWindow('frame',cv2.WINDOW_AUTOSIZE)
cap = cv2.VideoCapture(0)

if not cap.isOpened(): #True or False
    print("Videocap open failed!")
    cap.release()
    sys.exit()

# 카메라 프레임 크기 출력

w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
src2=np.ones((h,w,3),np.uint8)*255
src2=cv2.line(src2,(int(w*0.65),int(h*0.3)),(int(w*0.65),int(h*0.4)),(0,0,0),5,cv2.LINE_AA)
# cv2.imshow('src3',src2)
# print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
# print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

# 카메라 프레임 처리
while True:
    
    ret, frame1 = cap.read()
    key=cv2.waitKey(20)
    
    if not ret:
        print('Video read failed')
        break
    src1=cv2.flip(frame1,1)
    frame=cv2.addWeighted(src1,0.85,src2,0.15,0)
    cv2.setMouseCallback('frame', call_mouse, frame)
    if clilk == True:
        cv2.imshow('frame',frame)
    if key==27 or key==ord('q'):
        break
    elif key==ord('i'):
        frame=255-frame
        cv2.imshow('frame',frame)
#     elif key==ord('f'):
#         frame=cv2.flip(frame)
          

#     edge = cv2.Canny(frame, 50, 150)       
  

cap.release()
cv2.destroyAllWindows()

### 직선 검출

In [9]:

import cv2
import numpy as np

def line_detect(image):
  #       HSV
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  #     
  lowera = np.array([0, 0, 221])
  uppera = np.array([180, 30, 255])
  mask1 = cv2.inRange(hsv, lowera, uppera)
#   kernel = np.ones((3, 3), np.uint8)

#   #              （       ）
#   mask = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel) #   ：         ，       
#   mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)  #   ：         ，       

  #     
  edges = cv2.Canny(mask1, 50, 150, apertureSize=3)
  #     
  cv2.imshow("edges", edges)
  #                   ，      HoughLinesP()  ，               
  lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100,minLineLength=100,maxLineGap=10)
  print("lines=",lines)
  print("========================================================")
  i=1
  #                 
  for line in lines:
    # newlines1 = lines[:, 0, :]
    print("line["+str(i-1)+"]=",line)
    x1,y1,x2,y2 = line[0]  #        ，                  （x1,y1）(x2,y2)
    cv2.line(image,(x1,y1),(x2,y2),(0,0,255),2)   #      
    #       ，    
    x1 = float(x1)
    x2 = float(x2)
    y1 = float(y1)
    y2 = float(y2)
    print("x1=%s,x2=%s,y1=%s,y2=%s" % (x1, x2, y1, y2))
    if x2 - x1 == 0:
      print("      ")
      result=90
    elif y2 - y1 == 0 :
      print("      ")
      result=0
    else:
      #     
      k = -(y2 - y1) / (x2 - x1)
      #     ，           
      result = np.arctan(k) * 57.29577
      print("       ：" + str(result) + " ")
    i = i+1
  #           
  cv2.imshow("line_detect",image)
  return result

src = cv2.imread('./fig/straight-lines-intro.png')
cv2.namedWindow("input image", cv2.WINDOW_AUTOSIZE)
cv2.imshow("input image", src)
      
line_detect(src)
cv2.waitKey()
cv2.destroyAllWindows()


lines= [[[204 175 444  30]]

 [[ 35  23 275 168]]

 [[158  98 395 241]]

 [[ 41 269 301 112]]

 [[161 105 447 277]]

 [[ 36 277 237 156]]

 [[207 168 447  23]]

 [[ 35  28 255 161]]]
line[0]= [[204 175 444  30]]
x1=204.0,x2=444.0,y1=175.0,y2=30.0
       ：31.138967266831752 
line[1]= [[ 35  23 275 168]]
x1=35.0,x2=275.0,y1=23.0,y2=168.0
       ：-31.138967266831752 
line[2]= [[158  98 395 241]]
x1=158.0,x2=395.0,y1=98.0,y2=241.0
       ：-31.105748051975084 
line[3]= [[ 41 269 301 112]]
x1=41.0,x2=301.0,y1=269.0,y2=112.0
       ：31.12551202842472 
line[4]= [[161 105 447 277]]
x1=161.0,x2=447.0,y1=105.0,y2=277.0
       ：-31.022637053738546 
line[5]= [[ 36 277 237 156]]
x1=36.0,x2=237.0,y1=277.0,y2=156.0
       ：31.047517076951866 
line[6]= [[207 168 447  23]]
x1=207.0,x2=447.0,y1=168.0,y2=23.0
       ：31.138967266831752 
line[7]= [[ 35  28 255 161]]
x1=35.0,x2=255.0,y1=28.0,y2=161.0
       ：-31.154863991123854 
